In [38]:
from function import *
import numpy as np
import networkx as nx
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

In [39]:
# Using Random Data

# Number of data
N = 1000000
# Number of features
d = 5
# Control precision bit of the output 
precision = 3
# Size of the QUBO matrix
dim = d * (2 * precision)
# Degree of the polynomial regression
degree = 3

# Generate Random Data
X = np.random.rand(N, d)
Y = np.random.rand(N)

# Train-Test data splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# If you want to perform polynomial regression
X_train, d, dim = polynomialForm(X_train, d, dim, precision, degree)
X_test, _, _ = polynomialForm(X_test, d, dim, precision, degree) 

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
print(d, dim)

(800000, 56)
(200000, 56)
(800000,)
(200000,)
56 336


In [40]:
XtX = np.matmul(X_train.T, X_train) # Covariance matrix
XtY = np.matmul(X_train.T, Y_train) # Relation between Features and Y

print(XtX)
print(XtY)

[[800000.         400211.34778709 400561.96155841 ... 133365.54227412
  133309.67650569 200124.45096237]
 [400211.34778709 266836.70048387 200298.4829812  ...  66690.81487698
   66666.53363155 100133.26484683]
 [400561.96155841 200298.4829812  267212.67476394 ...  66735.22929684
   66717.4514864  100215.40547428]
 ...
 [133365.54227412  66690.81487698  66735.22929684 ...  53347.25302259
   49956.2706485   53246.31881766]
 [133309.67650569  66666.53363155  66717.4514864  ...  49956.2706485
   53246.31881766  66562.23022682]
 [200124.45096237 100133.26484683 100215.40547428 ...  53246.31881766
   66562.23022682 114301.44410927]]
[399756.79669628 200078.67320914 200100.99703946 199814.78172603
 199667.26981194 200104.32288001 133427.43628402 100150.94926186
  99987.97884474  99952.87051269 100143.72315232 133477.09393655
 100002.20778532  99908.96499093 100184.4444945  133206.47061738
  99812.19719619  99984.7929451  133062.17494389  99932.43307175
 133425.52127539 100073.34660863  66791.

In [41]:
# Creating the graph
G = nx.Graph()
G.add_edges_from([(i, j) for i in range(dim) for j in range(i + 1, dim)])

# The matrix where we add the objective and the constraint (QUBO matrix)
Q = defaultdict(int)

print(G)
print(Q)

Graph with 336 nodes and 56280 edges
defaultdict(<class 'int'>, {})


In [42]:
generateQuboMatrix(XtX, XtY, precision, Q, d)

### Neal Simulated Annealing

In [43]:
sampleset = sampling(Q)
distributions_neal, weights_neal = solve_neal(sampleset, dim, precision, d)

print(weights_neal)

[ 1.5   0.25 -1.   -0.75 -0.25 -0.75  0.25 -1.25 -0.25 -0.75  0.5   1.5
 -0.75  0.25  0.    0.   -0.25  0.5   0.75 -0.25 -0.75 -0.5   0.25  0.25
 -0.5  -0.25  0.25  0.75  0.    1.25  0.25  0.25 -1.    1.25  0.25 -0.25
 -0.75  0.25 -0.25 -0.25  0.25  0.75 -0.25 -0.25  0.5   0.    0.   -0.25
  0.25  1.   -0.75  0.75 -1.5   0.75 -0.25  0.75]


In [44]:
Y_train_pred_neal = np.dot(X_train, weights_neal)
Y_test_pred_neal = np.dot(X_test, weights_neal)

# Evaluate on training data
train_mse_neal = mean_squared_error(Y_train, Y_train_pred_neal)
train_mae_neal = mean_absolute_error(Y_train, Y_train_pred_neal)
train_r2_neal = r2_score(Y_train, Y_train_pred_neal)

# Evaluate on test data
test_mse_neal = mean_squared_error(Y_test, Y_test_pred_neal)
test_mae_neal = mean_absolute_error(Y_test, Y_test_pred_neal)
test_r2_neal = r2_score(Y_test, Y_test_pred_neal)

# Print the metrics
print("Neal Training Evaluation:")
print(f"MSE: {train_mse_neal:.5f}, MAE: {train_mae_neal:.5f}, R²: {train_r2_neal:.5f}")

print("\nNeal Testing Evaluation:")
print(f"MSE: {test_mse_neal:.5f}, MAE: {test_mae_neal:.5f}, R²: {test_r2_neal:.5f}")

Neal Training Evaluation:
MSE: 0.11657, MAE: 0.28285, R²: -0.39720

Neal Testing Evaluation:
MSE: 0.11688, MAE: 0.28339, R²: -0.40388


### Fixstar Amplify Simulated Annealing

In [45]:
distributions_fixstar, weights_fixstar = solve_fixstar(Q, dim, precision, d)

print(weights_fixstar)

[ 0.5   0.    0.5   0.5   0.   -0.5   0.   -0.25 -0.25 -0.5   0.75 -1.
 -0.75  0.25  0.75 -0.25 -0.5  -0.5   0.25  0.25  0.25  0.   -0.25  0.
  0.    0.25  0.5   0.    0.25 -0.25  0.25  0.    0.    0.5  -0.25 -0.75
  0.25  0.5   0.    0.25  0.25  0.25 -0.25 -0.25 -0.5  -0.5   0.    0.25
 -0.25  0.    0.25  0.75 -0.25  0.    0.    0.  ]


In [46]:
Y_train_pred_fixstar = np.dot(X_train, weights_fixstar)
Y_test_pred_fixstar = np.dot(X_test, weights_fixstar)

# Evaluate on training data
train_mse_fixstar = mean_squared_error(Y_train, Y_train_pred_fixstar)
train_mae_fixstar = mean_absolute_error(Y_train, Y_train_pred_fixstar)
train_r2_fixstar = r2_score(Y_train, Y_train_pred_fixstar) 

# Evaluate on test data
test_mse_fixstar = mean_squared_error(Y_test, Y_test_pred_fixstar)
test_mae_fixstar = mean_absolute_error(Y_test, Y_test_pred_fixstar)
test_r2_fixstar = r2_score(Y_test, Y_test_pred_fixstar)

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_fixstar:.5f}, MAE: {train_mae_fixstar:.5f}, R²: {train_r2_fixstar:.5f}")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_fixstar:.5f}, MAE: {test_mae_fixstar:.5f}, R²: {test_r2_fixstar:.5f}")    

Fixstar Training Evaluation:
MSE: 0.08535, MAE: 0.25211, R²: -0.02293

Fixstar Testing Evaluation:
MSE: 0.08517, MAE: 0.25178, R²: -0.02303


### Scikit Learn (Tradisional way)

In [47]:
import time
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

start_time = time.time() 

clf = LinearRegression()
clf.fit(X_train, Y_train)
print("Coefficients:", clf.coef_)

Y_train_pred_sklearn = clf.predict(X_train)
Y_test_pred_sklearn = clf.predict(X_test)

# Evaluate on training data
train_mse_sklearn = mean_squared_error(Y_train, Y_train_pred_sklearn)
train_mae_sklearn = mean_absolute_error(Y_train, Y_train_pred_sklearn)
train_r2_sklearn = r2_score(Y_train, Y_train_pred_sklearn) 

# Evaluate on test data
test_mse_sklearn = mean_squared_error(Y_test, Y_test_pred_sklearn)
test_mae_sklearn = mean_absolute_error(Y_test, Y_test_pred_sklearn)
test_r2_sklearn = r2_score(Y_test, Y_test_pred_sklearn)

end_time = time.time()  
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.5f} seconds\n")

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_sklearn:.5f}, MAE: {train_mae_sklearn:.5f}, R²: {train_r2_sklearn:.5f}")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_sklearn:.5f}, MAE: {test_mae_sklearn:.5f}, R²: {test_r2_sklearn:.5f}")  

Coefficients: [ 5.07307532e-19  1.47923253e-02 -1.24910518e-02  1.62349987e-02
  9.13023248e-03  2.75228160e-02 -1.71033919e-02  2.12985571e-02
 -3.49799242e-02 -8.42254643e-04 -4.60008288e-03  2.60743714e-02
  1.35762174e-02 -3.96422380e-02 -1.57915930e-02 -1.24423995e-02
 -2.13889727e-02 -1.77253940e-02  1.79472216e-02 -1.04945717e-02
 -4.24256509e-02  1.89597719e-03  9.31585934e-04  1.95790007e-02
  1.12021327e-02 -5.21946425e-03  1.59717587e-03 -1.69437156e-02
 -1.27362490e-03 -1.33838982e-02  4.20446522e-03  9.00181945e-03
  1.58888991e-02 -1.34982475e-02  8.43355177e-03  1.40860899e-03
 -2.33518761e-02 -2.03053667e-02  2.15486073e-02  1.97695688e-02
  9.22610289e-03  1.28466958e-02 -6.64861979e-03  2.48798363e-02
 -2.03438928e-02  1.58539438e-02  1.22318291e-02 -1.56886883e-02
  1.45630458e-03  2.37747959e-02  6.74872015e-03  5.32212675e-03
 -3.14806229e-02  2.14357700e-02 -7.27081063e-03  2.46908292e-02]
Execution time: 2.88099 seconds

Fixstar Training Evaluation:
MSE: 0.08343,

### Final Evaluation

In [48]:
data = {
    "Model": ["Neal", "Neal", "Fixstar", "Fixstar", "Sklearn", "Sklearn"],
    "Dataset": ["Train", "Test", "Train", "Test", "Train", "Test"],
    "MSE": [
        train_mse_neal, test_mse_neal,
        train_mse_fixstar, test_mse_fixstar,
        train_mse_sklearn, test_mse_sklearn
    ],
    "MAE": [
        train_mae_neal, test_mae_neal,
        train_mae_fixstar, test_mae_fixstar,
        train_mae_sklearn, test_mae_sklearn
    ],
    "R2 Score": [
        train_r2_neal, test_r2_neal,
        train_r2_fixstar, test_r2_fixstar,
        train_r2_sklearn, test_r2_sklearn
    ]
}

df_eval = pd.DataFrame(data)
df_eval

,Model,Dataset,MSE,MAE,R2 Score
0,Neal,Train,0.116575,0.282851,-0.397196
1,Neal,Test,0.116882,0.283389,-0.403882
2,Fixstar,Train,0.085348,0.252115,-0.022935
3,Fixstar,Test,0.085173,0.251781,-0.023026
4,Sklearn,Train,0.083429,0.250206,0.000071
5,Sklearn,Test,0.083259,0.249900,-0.000037
